----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content - Pricing - Chain + Streaming Constituents

This notebook demonstrates how to use the Pricing interface to 
- retrieve the constituents of a Chain instrument
- and then request Streaming data for the Constituents themselves

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
from refinitiv.data.content import pricing
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.platform.Definition object at 0x20ddbba9e20 {session_name='rdp'}>

### Define callbacks to display the streaming Constituents data
We will use these later to display the streaming data for the Constituent RICs

In [4]:
df = DataFrame()

# Function to initially populate Dataframe, once initial values received for all items
def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)

# Function to update dataframe, when we receive updates for individual items
def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)

### Get the Chain's constituent RICs
Define & snap the current constituents of the Chain and then display them

In [8]:
# Define a Chain object for the Dow Jones Index
dow = pricing.chain.Definition(name="0#.DJI").get_stream()
# We just want a snapshot of the current Index
dow.open(with_updates=False)
## Extract the consituent RICs and display
dow_constituents = dow.constituents
print(dow_constituents)

['.DJI', 'AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']


## Create Streams for the consitutuents
Define streams for all the consituents of the above chain, specifying two fields.   
Specify callbacks to display the field values:
- Call *create_dataframe* once we can received the intial response for all the constituents
- Call *update_dataframe* as and when we receive field update for any of the consituents

In [9]:
# Define our Streaming Price object for all the consituents
pricing_stream = pricing.Definition(universe=dow_constituents, fields=["BID", "ASK"]).get_stream()
# Specify the callback to create and display a dataframe 
# on_complete is invoked once we receive the initial image for all our instruments
pricing_stream.on_complete(create_dataframe)
#Specify the callback to display subsequent updates as and when received
pricing_stream.on_update(update_dataframe)


<refinitiv.data.content.pricing.Stream object at 0x20dff6213d0 {name='['.DJI', 'AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']'}>

## Open the streams for all our constituents

In [10]:
# Request all instrument from server and open streams
pricing_stream.open()
# The callbacks we declared earlier should be invoked as and when we receive server responses
# i.e. create_dataframe will be called once we have the initial values for each instrument
# after which, update_dataframe will be invoked as and when any instrument's BID/ASK fields are updated

,BID,ASK
.DJI,35667.95,35868.11
AAPL.OQ,158.96,159.2
AMGN.OQ,202.0,203.04
AXP.N,0.0,0.0
BA.N,0.0,0.0
CAT.N,0.0,0.0
CRM.N,0.0,0.0
CSCO.OQ,54.83,55.1
CVX.N,0.0,0.0
DIS.N,0.0,0.0


## Close the streams for the constituents

In [11]:
pricing_stream.close()

<OpenState.Closed: 3>

## Close the session

In [13]:
rd.close_session()